In [99]:
import pandas as pd
import os
# Define file paths and their corresponding names
files = {
    'nifty': 'Niftweekly.csv',
    'sensex': 'SensexWeekly.csv',
    'Midcap150': 'Midcap150Weekly.csv',
    'MidcapBSE': 'MidCapWeekly.csv',
    'Next50': 'Next50Weekly.csv',
    'Nifty100': 'Nifty100Weekly.csv',
    'SmallCap100': 'SmallCap100Weekly.csv'
}
# Verify files exist
missing_files = []
for name, file in files.items():
    if not os.path.exists(file):
        missing_files.append(file)
        print(f"Warning: {file} not found!")

if missing_files:
    print("\nMissing files:")
    for file in missing_files:
        print(f"- {file}")
    raise FileNotFoundError("Some required files are missing!")

# Dictionary to store all DataFrames
dfs = {}

# Load all data files
for name, file in files.items():
    df = pd.read_csv(file)
    
    # Apply transformations
    df.rename(columns={'Change %': 'Change'}, inplace=True)
    df.drop(columns=['Open', 'High', 'Low', 'Vol.'], inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
    df['Change'] = df['Change'].str.replace('%', '', regex=False).astype(float)
    df['Price'] = df['Price'].str.replace(',', '', regex=False).astype(float)
    df['Year'] = df['Date'].dt.year
    
    # Sort by date
    df = df.sort_values('Date', ascending=True).reset_index(drop=True)
    
    # Store in dictionary
    dfs[name] = df
# Access DataFrames using the dictionary
print(f"\nAll {len(dfs)} files loaded successfully!")



All 7 files loaded successfully!


In [100]:

# Function to simulate strategies per year
def simulate_sip_strategies(df, index_name, fall_threshold=-3):
    years = df['Year'].unique()
    current_price = df.iloc[-1]['Price']  # Use latest price in dataset
    sip_logs = []

    for year in years:
        # SIP strategy
        sip_units, sip_invested = 0, 0
        for month in range(1, 13):
            month_data = df[(df['Year'] == year) & (df['Date'].dt.month == month)]
            if not month_data.empty:
                row = month_data.iloc[0]
                units = 10000 / row['Price']
                sip_units += units
                sip_invested += 10000
                sip_logs.append({
                    'Strategy': 'SIP',
                    'Date': row['Date'],
                    'Price': row['Price'],
                    'Amount Invested': 10000,
                    'Units': units,
                    'Current Value': units * current_price,
                    'Running Total Investment': sip_invested,
                    'Running Corpus Value': sip_units * current_price,
                    'Index': index_name,
                    'Year': year
                })

    return pd.DataFrame(sip_logs)
def simulate_dip_strategies(df, index_name, fall_threshold=-3):
    years = df['Year'].unique()
    current_price = df.iloc[-1]['Price']  # Use latest price in dataset
    dip_logs = []
    for year in years:
        # DIP strategy
        dip_data = df[(df['Year'] == year) & (df['Change'] < fall_threshold)]
        dip_units, dip_invested = 0, 0
        dip_count = 0

        for _, row in dip_data.iterrows():
            if dip_count >= 12:
                break
            units = 10000 / row['Price']
            dip_units += units
            dip_invested += 10000
            dip_logs.append({
                'Strategy': 'DIP',
                'Date': row['Date'],
                'Price': row['Price'],
                'Amount Invested': 10000,
                'Units': units,
                'Current Value': units * current_price,
                'Running Total Investment': dip_invested,
                'Running Corpus Value': dip_units * current_price,
                'Index': index_name,
                'Year': year
            })
            dip_count += 1

        if dip_count < 12:
            dec_data = df[(df['Year'] == year) & (df['Date'].dt.month == 12)]
            if not dec_data.empty:
                dec_row = dec_data.iloc[-1]
                for _ in range(12 - dip_count):
                    units = 10000 / dec_row['Price']
                    dip_units += units
                    dip_invested += 10000
                    dip_logs.append({
                        'Strategy': 'DEC_FILL',
                        'Date': dec_row['Date'],
                        'Price': dec_row['Price'],
                        'Amount Invested': 10000,
                        'Units': units,
                        'Current Value': units * current_price,
                        'Running Total Investment': dip_invested,
                        'Running Corpus Value': dip_units * current_price,
                        'Index': index_name,
                        'Year': year
                    })
    return pd.DataFrame(dip_logs)

# Dictionary to store all logs
logs = {}

# Generate full logs for all indices
fall_threshold = -3  # Configurable

# Process each DataFrame
for index_name, df in dfs.items():
    # Generate SIP and DIP logs
    sip_logs = simulate_sip_strategies(df, index_name, fall_threshold)
    dip_logs = simulate_dip_strategies(df, index_name, fall_threshold)
    
    # Store logs
    logs[f"{index_name}_sip"] = sip_logs
    logs[f"{index_name}_dip"] = dip_logs
    
    # Calculate metrics
    sip_total_units = sip_logs['Units'].sum()
    sip_final_price = sip_logs['Price'].iloc[-1]
    dip_total_units = dip_logs['Units'].sum()
    dip_final_price = dip_logs['Price'].iloc[-1]
    
    # Print results
    print(f"\n{index_name.upper()} SIP Strategy:")
    print(f"Total Units Accumulated: {sip_total_units:.2f}")
    print(f"Final Corpus Value: ₹{sip_final_price*sip_total_units:,.2f}")
    
    print(f"\n{index_name.upper()} DIP Strategy:")
    print(f"Total Units Accumulated: {dip_total_units:.2f}")
    print(f"Final Corpus Value: ₹{dip_final_price*dip_total_units:,.2f}")

# Export all results to Excel
with pd.ExcelWriter("dip_vs_sip_analysis.xlsx", engine="openpyxl") as writer:
    for log_name, log_df in logs.items():
        sheet_name = log_name.replace('_', ' ').title()
        log_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("\nAnalysis complete! Results exported to dip_vs_sip_analysis.xlsx")



NIFTY SIP Strategy:
Total Units Accumulated: 856.73
Final Corpus Value: ₹21,142,222.95

NIFTY DIP Strategy:
Total Units Accumulated: 814.46
Final Corpus Value: ₹19,550,952.12

SENSEX SIP Strategy:
Total Units Accumulated: 772.79
Final Corpus Value: ₹61,670,989.37

SENSEX DIP Strategy:
Total Units Accumulated: 737.78
Final Corpus Value: ₹58,062,980.50

MIDCAP150 SIP Strategy:
Total Units Accumulated: 80.34
Final Corpus Value: ₹1,742,521.66

MIDCAP150 DIP Strategy:
Total Units Accumulated: 75.00
Final Corpus Value: ₹1,331,311.73

MIDCAPBSE SIP Strategy:
Total Units Accumulated: 409.06
Final Corpus Value: ₹18,856,246.98

MIDCAPBSE DIP Strategy:
Total Units Accumulated: 375.20
Final Corpus Value: ₹14,479,789.29

NEXT50 SIP Strategy:
Total Units Accumulated: 498.01
Final Corpus Value: ₹33,861,012.72

NEXT50 DIP Strategy:
Total Units Accumulated: 447.30
Final Corpus Value: ₹25,524,226.60

NIFTY100 SIP Strategy:
Total Units Accumulated: 339.19
Final Corpus Value: ₹8,696,455.08

NIFTY100 DIP 